# XOR - *Forward propagation* y *backpropagation*

Statistical Learning II

Rodrigo Chang

19000625

***
## Librerías y funciones de activación y bias

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Activación sigmoide
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Activación de escalón
def heaviside(x):
    return (x >= 0).astype(np.float)

# Función de activación ReLU
def ReLU(x):
    return np.maximum(0., x)

def ReLU_gradient(x):
    return (x > 0).astype(np.float)

def linearAct(x):
    return x

# Función para añadir el término de bias a la primera fila
# x tiene shape (n, k)
# Devuelve matriz con tamaño (n+1, k)
def addBias(x):
    return np.vstack((np.ones(x.shape[1]), x))

# Función para quitar el término de bias de la primera fila
# x tiene shape (n+1, k)
# Devuelve matriz con tamaño (n, k)
def removeBias(x):
    return x[1:, :].reshape(-1, x.shape[1])

In [41]:
sigmoid(np.array([-1, 0., 1., 2.]))

array([0.26894142, 0.5       , 0.73105858, 0.88079708])

In [5]:
ReLU(np.array([-1, 0., 1., 2.]))

array([0., 0., 1., 2.])

In [6]:
ReLU_gradient(np.array([-1, 0., 1., 2.]))

array([0., 0., 1., 1.])

***
## Conjunto de datos de entrenamiento
Definimos el conjunto de datos de entrenamiento. En este caso, queremos que la red neuronal aprenda la función XOR, por lo tanto, utilizaremos dos entradas y sus respectivas salidas conocidas.

In [76]:
Xtrain = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
Xtrain

array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [1., 1.]])

In [77]:
Ytrain = np.array([0., 1., 1., 0.]).reshape(-1, 1)
Ytrain

array([[0.],
       [1.],
       [1.],
       [0.]])

In [78]:
Xtrain.T, Ytrain.T

(array([[0., 0., 1., 1.],
        [0., 1., 0., 1.]]), array([[0., 1., 1., 0.]]))

***
## Matrices de pesos sinápticos
Definimos los pesos sinápticos iniciales de la red neuronal.

In [90]:
# Capa oculta 1
k1 = 3
Theta1 = np.random.normal(scale=0.1, size=(k1, 3))
Theta1

array([[ 0.15415817, -0.22765972,  0.00147032],
       [ 0.08778565, -0.00696515,  0.20408783],
       [ 0.01326903,  0.01138651, -0.08460882]])

In [91]:
# Capa oculta 2
k2 = 2
Theta2 = np.random.normal(scale=0.1, size=(k2, k1+1))
Theta2

array([[ 0.16646132, -0.23252446,  0.12473551, -0.14228832],
       [-0.04169177,  0.03146323, -0.1531307 , -0.00371119]])

In [92]:
# Capa de salida
Theta3 = np.random.normal(scale=0.1, size=(1, 3))
Theta3

array([[0.0512652 , 0.03846694, 0.12638179]])

***
## *Forward propagation* y cómputo de la salida

In [107]:
Xtrain.T, Xtrain.T.shape

(array([[0., 0., 1., 1.],
        [0., 1., 0., 1.]]), (2, 4))

In [73]:
def forwardProp(X, Theta1, Theta2, Theta3):
    # Primera capa oculta
    z_2 = np.matmul(Theta1, addBias(X))
    a_2 = ReLU(z_2)
    #print("a_2: ", a_2, "\n")
    # Segunda capa oculta
    z_3 = np.matmul(Theta2, addBias(a_2))
    a_3 = ReLU(z_3)
    #print("a_3: ", a_3, "\n")
    # Capa de salida
    z_4 = np.matmul(Theta3, addBias(a_3))
    a_4 = linearAct(z_4)
    return a_4

In [74]:
forwardProp(Xtrain.T, Theta1, Theta2, Theta3)

array([[-0.09787489, -0.09787489, -0.09787489, -0.09787489]])

In [75]:
forwardProp(Xtrain.T, Theta1_trained, Theta2_trained, Theta3_trained)

array([[0.3708145 , 0.37081451, 0.3708149 , 0.37081491]])

In [72]:
forwardProp(np.array([100., 50.]).reshape(-1, 1), Theta1, Theta2, Theta3)

array([[-0.09787489]])

***
## *Backpropagation* de un ejemplo
Propagación del error hacia atrás utilizando el ejemplo $(0, 1)$ cuya salida debería ser $1$.

In [17]:
Xtest = Xtrain[1, :].reshape(-1, 1)
Ytest = Ytrain[1].reshape(-1,1)
Xtest, Ytest

(array([[0.],
        [1.]]), array([[1.]]))

In [45]:
# Propagamos hacia adelante el ejemplo
a_1 = addBias(Xtest)
# Primera capa oculta
z_2 = np.matmul(Theta1, a_1)
a_2 = ReLU(z_2)
# Segunda capa oculta
a_2 = addBias(a_2)
z_3 = np.matmul(Theta2, a_2)
a_3 = ReLU(z_3)
# Capa de salida
a_3 = addBias(a_3)
z_4 = np.matmul(Theta3, a_3)
a_4 = linearAct(z_4)

## Backpropagation
# Computamos el error en la capa de salida: d_4
d_4 = a_4 - Ytest
d_4

array([[-1.09787489]])

In [25]:
# Propagamos hacia atrás
d_3 = np.matmul(Theta3.T[1:], d_4) * ReLU_gradient(z_3)
d_3

array([[-0.],
       [-0.]])

In [26]:
# Propagamos hacia atrás
d_2 = np.matmul(Theta2.T[1:, :], d_3) * ReLU_gradient(z_2)
d_2

array([[0.],
       [0.],
       [0.]])

### Matrices de cambio
Construimos las matrices de cambio, para la actualización de las matrices de pesos sinápticos

In [27]:
Delta3 = np.matmul(d_4, a_3.T)
Delta3

array([[-1.09787489,  0.        ,  0.        ]])

In [28]:
Delta2 = np.matmul(d_3, a_2.T)
Delta2

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [29]:
Delta1 = np.matmul(d_2, a_1.T)
Delta1

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

***
## Función de costo

In [84]:
np.power(forwardProp(Xtrain.T, Theta1, Theta2, Theta3).T - Ytrain, 2).mean() / 2

0.30372719291614736

In [85]:
Xtrain.shape[0]

4

In [93]:
def nnCosto(X, Y, Theta1, Theta2, Theta3):
    
    # Número de ejemplos
    m = X.shape[0]
    
    # Definir el costo del conjunto con el MSE
    error = forwardProp(X.T, Theta1, Theta2, Theta3).T - Y
    # Función de costo
    J = np.mean(np.power(error, 2)) / 2
    
    # Cómputo de los gradientes 
    Delta1 = np.zeros_like(Theta1)
    Delta2 = np.zeros_like(Theta2)
    Delta3 = np.zeros_like(Theta3)
    
    # Backpropagation para cada ejemplo
    for i in range(m):
        # Obtenemos el ejemplo i
        Xi = X[i, :].reshape(-1, 1)
        # Propagamos hacia adelante el ejemplo
        a_1 = addBias(Xi)
        # Primera capa oculta
        z_2 = np.matmul(Theta1, a_1)
        a_2 = ReLU(z_2)
        # Segunda capa oculta
        a_2 = addBias(a_2)
        z_3 = np.matmul(Theta2, a_2)
        a_3 = ReLU(z_3)
        # Capa de salida
        a_3 = addBias(a_3)
        z_4 = np.matmul(Theta3, a_3)
        a_4 = linearAct(z_4)
    
        # Computamos el error en la capa de salida y propagamos hacia atrás
        d_4 = a_4 - Ytest
        d_3 = np.matmul(Theta3.T[1:], d_4) * ReLU_gradient(z_3)
        d_2 = np.matmul(Theta2.T[1:, :], d_3) * ReLU_gradient(z_2)
        
        # Matrices de cambio
        Delta3 += np.matmul(d_4, a_3.T)
        Delta2 += np.matmul(d_3, a_2.T)
        Delta1 += np.matmul(d_2, a_1.T)
    
    Theta1_grad = Delta1 / m
    Theta2_grad = Delta2 / m
    Theta3_grad = Delta3 / m

    return J, Theta1_grad, Theta2_grad, Theta3_grad

In [105]:
nnCosto(Xtrain, Ytrain, Theta1, Theta2, Theta3)

(0.22277226973590006, array([[ 0.00421663,  0.        ,  0.00210715],
        [-0.00452077, -0.0022588 , -0.00225909],
        [ 0.00257994,  0.00128909,  0.        ]]), array([[-0.03624286, -0.00280885, -0.00675172, -0.00034375],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[-0.94218203, -0.16048004,  0.        ]]))

***
## Implementación del gradiente en descenso para entrenamiento

In [99]:
def entrenarNN(X, Y, k1=3, k2=2, epochs=100, lr=0.001, statusRate=50):
    
    # Número de features
    n = X.shape[1]
    
    # Matrices de pesos sinápticos aleatorias
    Theta1 = np.random.normal(scale=0.05, size=(k1, n+1))
    Theta2 = np.random.normal(scale=0.05, size=(k2, k1+1))
    Theta3 = np.random.normal(scale=0.05, size=(1, 3))
    
    # Iterar sobre el número de epochs
    for j in range(1, epochs+1):
        # Obtener el costo y los gradientes
        J, Theta1_grad, Theta2_grad, Theta3_grad = nnCosto(X, Y, Theta1, Theta2, Theta3)
        
        if (j % statusRate == 0):
            print("Iteración: %d,\tCosto: %0.4f" % (j, J))
        
        # Actualizar los pesos sinápticos
        Theta1 += -lr * Theta1_grad
        Theta2 += -lr * Theta2_grad
        Theta3 += -lr * Theta3_grad

    return Theta1, Theta2, Theta3

In [108]:
Theta1_trained, Theta2_trained, Theta3_trained = entrenarNN(Xtrain, Ytrain, 
                                                            epochs=5000, lr=0.0001, statusRate=500)

Iteración: 500,	Costo: 0.2300
Iteración: 1000,	Costo: 0.2096
Iteración: 1500,	Costo: 0.1922
Iteración: 2000,	Costo: 0.1776
Iteración: 2500,	Costo: 0.1653
Iteración: 3000,	Costo: 0.1552
Iteración: 3500,	Costo: 0.1469
Iteración: 4000,	Costo: 0.1402
Iteración: 4500,	Costo: 0.1350
Iteración: 5000,	Costo: 0.1310


In [109]:
forwardProp(Xtrain.T, Theta1_trained, Theta2_trained, Theta3_trained)

array([[0.39006019, 0.39005634, 0.39006708, 0.39005945]])

In [102]:
Theta1_trained, Theta2_trained, Theta3_trained = entrenarNN(Xtrain, Ytrain, 
                                                            epochs=5000, lr=0.0001, statusRate=100)

Iteración: 100,	Costo: 0.2355
Iteración: 200,	Costo: 0.2310
Iteración: 300,	Costo: 0.2267
Iteración: 400,	Costo: 0.2224
Iteración: 500,	Costo: 0.2183
Iteración: 600,	Costo: 0.2144
Iteración: 700,	Costo: 0.2105
Iteración: 800,	Costo: 0.2068
Iteración: 900,	Costo: 0.2032
Iteración: 1000,	Costo: 0.1997
Iteración: 1100,	Costo: 0.1964
Iteración: 1200,	Costo: 0.1931
Iteración: 1300,	Costo: 0.1899
Iteración: 1400,	Costo: 0.1869
Iteración: 1500,	Costo: 0.1840
Iteración: 1600,	Costo: 0.1811
Iteración: 1700,	Costo: 0.1784
Iteración: 1800,	Costo: 0.1757
Iteración: 1900,	Costo: 0.1731
Iteración: 2000,	Costo: 0.1707
Iteración: 2100,	Costo: 0.1683
Iteración: 2200,	Costo: 0.1660
Iteración: 2300,	Costo: 0.1638
Iteración: 2400,	Costo: 0.1617
Iteración: 2500,	Costo: 0.1596
Iteración: 2600,	Costo: 0.1576
Iteración: 2700,	Costo: 0.1558
Iteración: 2800,	Costo: 0.1539
Iteración: 2900,	Costo: 0.1522
Iteración: 3000,	Costo: 0.1505
Iteración: 3100,	Costo: 0.1489
Iteración: 3200,	Costo: 0.1474
Iteración: 3300,	

In [110]:
forwardProp(Xtrain.T, Theta1_trained, Theta2_trained, Theta3_trained)

array([[0.39006019, 0.39005634, 0.39006708, 0.39005945]])

In [111]:
Theta1_trained

array([[ 0.03480973, -0.02545688, -0.02837511],
       [-0.07013853,  0.06595636, -0.02542802],
       [ 0.04686075,  0.03639778, -0.06366123]])

In [112]:
Theta2_trained

array([[ 0.05497828, -0.05486433,  0.01006399,  0.08805007],
       [-0.0012665 ,  0.00521325, -0.044771  , -0.0633429 ]])

In [113]:
Theta3_trained

array([[0.38997452, 0.00149783, 0.10637533]])